In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 44.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=c6a04cc149ae884e47ae737c2558ab89ceda11dd4e16bf0a508577a916b26c00
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
#import and create sparksession object
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('rc').getOrCreate()

In [ ]:
#import the required functions and libraries
from pyspark.sql.functions import *

In [ ]:
import sys
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions  import *
from pyspark.sql.types import *

In [ ]:
# define the configurations for this Spark program
conf = SparkConf().setMaster("local[*]").setAppName("Clothing")
conf.set("spark.executor.memory", "6G")
conf.set("spark.driver.memory", "2G")
conf.set("spark.executor.cores", "4")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.set("spark.default.parallelism", "4")

# create a Spark Session instead of a Spark Context
spark = SparkSession.builder \
    .config(conf = conf) \
  .appName("spark session example") \
  .getOrCreate()
sc = spark.sparkContext

In [ ]:
#load the dataset and create sprk dataframe
df=spark.read.csv('/content/Ratings.csv',inferSchema=True,header=True)

In [ ]:
df=df.dropna()

In [ ]:
df.show()

+-------+-----------+------+
|User_ID|Clothing_ID|Rating|
+-------+-----------+------+
|      0|        767|     4|
|      1|       1080|     5|
|      2|       1077|     3|
|      3|       1049|     5|
|      4|        847|     5|
|      5|       1080|     2|
|      6|        858|     5|
|      7|        858|     4|
|      8|       1077|     5|
|      9|       1077|     5|
|     10|       1077|     3|
|     11|       1095|     5|
|     12|       1095|     5|
|     13|        767|     5|
|     14|       1077|     3|
|     15|       1065|     4|
|     16|       1065|     3|
|     17|        853|     5|
|     18|       1120|     5|
|     19|       1077|     5|
+-------+-----------+------+
only showing top 20 rows



In [ ]:
df.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Clothing_ID: integer (nullable = true)
 |-- Rating: integer (nullable = true)



In [ ]:
#import ALS recommender function from pyspark ml library
from pyspark.ml.recommendation import ALS

In [ ]:
#Training the recommender model using train datatset
als = ALS(maxIter=10, regParam=0.01, userCol="User_ID", itemCol="Clothing_ID", ratingCol="Rating",nonnegative=True,coldStartStrategy="drop")

In [ ]:
model=als.fit(df)

In [ ]:
info_df=spark.read.csv('/content/Womens Clothing E-Commerce Reviews.csv',inferSchema=True,header=True)

In [ ]:
info_df=info_df.dropna()

In [ ]:
ratings = df.filter(col('User_ID')==808)


In [ ]:
ratings.show()

+-------+-----------+------+
|User_ID|Clothing_ID|Rating|
+-------+-----------+------+
|    808|        886|     3|
+-------+-----------+------+



#recommendForUserSubset

In [ ]:
ID = [[808]]
# convert this into a dataframe so that it can be passed into the recommendForUserSubset
functiondf = sc.parallelize(ID).toDF(['User_ID'])
num_rec = 10

In [ ]:
functiondf.show()

+-------+
|User_ID|
+-------+
|    808|
+-------+



In [ ]:
recommendations = model.recommendForUserSubset(functiondf , num_rec)
recommendations.collect()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


[Row(User_ID=808, recommendations=[Row(Clothing_ID=501, rating=8.273388862609863), Row(Clothing_ID=631, rating=8.174712181091309), Row(Clothing_ID=668, rating=8.108043670654297), Row(Clothing_ID=1132, rating=8.00462818145752), Row(Clothing_ID=447, rating=7.6829304695129395), Row(Clothing_ID=417, rating=7.3790411949157715), Row(Clothing_ID=293, rating=7.25611686706543), Row(Clothing_ID=539, rating=7.021347522735596), Row(Clothing_ID=509, rating=6.978665351867676), Row(Clothing_ID=684, rating=6.894820213317871)])]

In [ ]:
recommended_clothing = [recommendations.collect()[0]['recommendations'][x]['Clothing_ID'] for x in range(0,num_rec)]
recommended_clothing

[501, 631, 668, 1132, 447, 417, 293, 539, 509, 684]

In [ ]:
# convert the recommended_ISBN list into a dataframe so that it can be joined with books_df
rec_df = spark.createDataFrame(recommended_clothing, IntegerType())

In [ ]:
rec_df.show()

+-----+
|value|
+-----+
|  501|
|  631|
|  668|
| 1132|
|  447|
|  417|
|  293|
|  539|
|  509|
|  684|
+-----+



In [ ]:
df2=rec_df.join(info_df,rec_df.value == info_df.Clothing_ID)

In [ ]:
df2.show()

+-----+-----------+---+--------------------+--------------------+--------------------+---------------+-----------------------+--------------+---------------+----------+
|value|Clothing_ID|Age|               Title|         Review Text|              Rating|Recommended IND|Positive Feedback Count| Division Name|Department Name|Class Name|
+-----+-----------+---+--------------------+--------------------+--------------------+---------------+-----------------------+--------------+---------------+----------+
|  501|        501| 46|Lune embroidered ...|Really striking. ...|                   5|              1|                      0|     Initmates|       Intimate|     Sleep|
|  501|        501| 60|      Buy this robe!|"Buy this robe be...| but i tried on t...|              5|                      1|             0|      Initmates|  Intimate|
|  631|        631| 27|   Beautiful chemise|This is a very so...|                   4|              1|                      0|General Petite|       Intimat

In [ ]:
df2.select(col('Clothing_ID'),col('Division Name'),col('Department Name'),col('Class Name')).show()

+-----------+--------------+---------------+----------+
|Clothing_ID| Division Name|Department Name|Class Name|
+-----------+--------------+---------------+----------+
|        501|     Initmates|       Intimate|     Sleep|
|        501|             0|      Initmates|  Intimate|
|        631|General Petite|       Intimate|    Lounge|
|       1132|General Petite|        Jackets| Outerwear|
|       1132|General Petite|        Jackets| Outerwear|
|       1132|General Petite|        Jackets| Outerwear|
|        447|       General|        Bottoms|    Shorts|
|        417|       General|        Bottoms|    Shorts|
|        417|       General|        Bottoms|    Shorts|
|        417|       General|        Bottoms|    Shorts|
|        417|       General|        Bottoms|    Shorts|
|        417|       General|        Bottoms|    Shorts|
|        417|       General|        Bottoms|    Shorts|
|        417|       General|        Bottoms|    Shorts|
|        417|       General|        Bottoms|    

#recommendForAllUsers

In [ ]:
recommendations1 = model.recommendForAllUsers(5)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
recommendations1.show(5)

+-------+--------------------+
|User_ID|     recommendations|
+-------+--------------------+
|      1|[{548, 25.935865}...|
|      3|[{359, 7.872712},...|
|      5|[{548, 10.374346}...|
|      6|[{452, 11.24725},...|
|      9|[{147, 27.374147}...|
+-------+--------------------+
only showing top 5 rows



In [ ]:
nrecommendations = recommendations1\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('User_ID', col("rec_exp.Clothing_ID"), col("rec_exp.Rating"))

nrecommendations.limit(10).show()

+-------+-----------+---------+
|User_ID|Clothing_ID|   Rating|
+-------+-----------+---------+
|      1|        548|25.935865|
|      1|        268|23.036076|
|      1|        147|22.714087|
|      1|        102|21.344572|
|      1|        328| 20.73505|
|      3|        359| 7.872712|
|      3|         34|7.8214884|
|      3|        335|7.7897143|
|      3|        357|7.7225404|
|      3|         98|7.4652815|
+-------+-----------+---------+



In [ ]:
nrecommendations.join(info_df, on='Clothing_ID').filter('User_ID = 1').show()

+-----------+-------+---------+---+--------------------+--------------------+------+---------------+-----------------------+-------------+---------------+----------+
|Clothing_ID|User_ID|   Rating|Age|               Title|         Review Text|Rating|Recommended IND|Positive Feedback Count|Division Name|Department Name|Class Name|
+-----------+-------+---------+---+--------------------+--------------------+------+---------------+-----------------------+-------------+---------------+----------+
|        268|      1|23.036076| 53|     Blissfully soft|This robe is one ...|     5|              1|                     12|    Initmates|       Intimate|     Sleep|
|        268|      1|23.036076| 39|             So soft|I love this robe,...|     5|              1|                      0|    Initmates|       Intimate|     Sleep|
|        147|      1|22.714087| 36|Sleeping in comfo...|George venson of ...|     5|              1|                     11|    Initmates|       Intimate|     Sleep|
|   

#recommendForAllItems

In [ ]:
recommendations2=model.recommendForAllItems(5)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
recommendations2.show(10)

+-----------+--------------------+
|Clothing_ID|     recommendations|
+-----------+--------------------+
|          1|[{7280, 27.415129...|
|          3|[{2210, 26.29809}...|
|          5|[{12770, 26.24665...|
|          6|[{12770, 27.79749...|
|          9|[{2210, 31.425873...|
|         12|[{9574, 36.11039}...|
|         13|[{23192, 25.26461...|
|         15|[{6120, 25.312954...|
|         16|[{1720, 24.95262}...|
|         17|[{2160, 37.054543...|
+-----------+--------------------+
only showing top 10 rows



In [ ]:
recommendations2 = recommendations2\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('Clothing_ID', col("rec_exp.User_ID"), col("rec_exp.Rating"))

recommendations2.limit(10).show()

+-----------+-------+---------+
|Clothing_ID|User_ID|   Rating|
+-----------+-------+---------+
|          1|   7280|27.415129|
|          1|   6840|27.415129|
|          1|   6120|27.415129|
|          1|   1720|27.415129|
|          1|   6140|27.415129|
|          3|   2210| 26.29809|
|          3|    460| 26.29809|
|          3|   2160| 26.29809|
|          3|   2130| 26.29809|
|          3|   2240| 26.29809|
+-----------+-------+---------+



In [ ]:
recommendations2.join(info_df, on='Clothing_ID').filter('Clothing_ID = 564').show()

+-----------+-------+---------+---+-------------+--------------------+------+---------------+-----------------------+-------------+---------------+----------+
|Clothing_ID|User_ID|   Rating|Age|        Title|         Review Text|Rating|Recommended IND|Positive Feedback Count|Division Name|Department Name|Class Name|
+-----------+-------+---------+---+-------------+--------------------+------+---------------+-----------------------+-------------+---------------+----------+
|        564|   9492|18.396559| 53|Flapper-esque|Oh this is so muc...|     5|              1|                      4|      General|          Trend|     Trend|
|        564|   4711|18.396559| 53|Flapper-esque|Oh this is so muc...|     5|              1|                      4|      General|          Trend|     Trend|
|        564|   5032|18.396559| 53|Flapper-esque|Oh this is so muc...|     5|              1|                      4|      General|          Trend|     Trend|
|        564|  21850|18.396559| 53|Flapper-esq

#recommendForItemSubset

In [ ]:
ratings1 = df.filter(col('Clothing_ID')==564)


In [ ]:
ratings1.show()

+-------+-----------+------+
|User_ID|Clothing_ID|Rating|
+-------+-----------+------+
|  14544|        564|     5|
+-------+-----------+------+



In [ ]:
ID1 = [[564]]
# convert this into a dataframe so that it can be passed into the recommendForUserSubset
functiondf1 = sc.parallelize(ID1).toDF(['Clothing_ID'])
num_rec1 = 10

In [ ]:
functiondf1.show()

+-----------+
|Clothing_ID|
+-----------+
|        564|
+-----------+



In [ ]:
recommendations3 = model.recommendForItemSubset(functiondf1 , num_rec1)
recommendations3.collect()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


[Row(Clothing_ID=564, recommendations=[Row(User_ID=22487, rating=18.39655876159668), Row(User_ID=4875, rating=18.39655876159668), Row(User_ID=18646, rating=18.39655876159668), Row(User_ID=9492, rating=18.39655876159668), Row(User_ID=9574, rating=18.39655876159668), Row(User_ID=9586, rating=18.39655876159668), Row(User_ID=5032, rating=18.39655876159668), Row(User_ID=21850, rating=18.39655876159668), Row(User_ID=4711, rating=18.39655876159668), Row(User_ID=14487, rating=18.39655876159668)])]

In [ ]:
recommended_clothing1 = [recommendations3.collect()[0]['recommendations'][x]['User_ID'] for x in range(0,num_rec1)]
recommended_clothing1

[22487, 4875, 18646, 9492, 9574, 9586, 5032, 21850, 4711, 14487]

In [ ]:
# convert the recommended_ISBN list into a dataframe so that it can be joined with books_df
rec_df1 = spark.createDataFrame(recommended_clothing1, IntegerType())

In [ ]:
rec_df1.show()

+-----+
|value|
+-----+
|22487|
| 4875|
|18646|
| 9492|
| 9574|
| 9586|
| 5032|
|21850|
| 4711|
|14487|
+-----+



In [ ]:
User_df=spark.read.csv('/content/User_info.csv',inferSchema=True,header=True)

In [ ]:
df3 = rec_df1.join(User_df,rec_df1.value == User_df.User_ID)

In [ ]:
df3.show()

+-----+-------+---+-----------+---------------+-----------------------+
|value|User_ID|Age|Review Text|Recommended IND|Positive Feedback Count|
+-----+-------+---+-----------+---------------+-----------------------+
|22487|  22487| 77|       null|              1|                      1|
| 4875|   4875| 36|       null|              1|                      0|
|18646|  18646| 66|       null|              1|                      1|
| 9492|   9492| 44|       null|              1|                      1|
| 9574|   9574| 42|       null|              1|                      1|
| 9586|   9586| 45|       null|              1|                      6|
| 5032|   5032| 39|       null|              1|                      2|
|21850|  21850| 36|       null|              1|                      0|
| 4711|   4711| 62|       null|              1|                      0|
|14487|  14487| 39|       null|              1|                      0|
+-----+-------+---+-----------+---------------+-----------------

In [ ]:
df3.select(col('User_ID'),col('Age'),col('Recommended IND'),col('Positive Feedback Count')).show()

+-------+---+---------------+-----------------------+
|User_ID|Age|Recommended IND|Positive Feedback Count|
+-------+---+---------------+-----------------------+
|  22487| 77|              1|                      1|
|   4875| 36|              1|                      0|
|  18646| 66|              1|                      1|
|   9492| 44|              1|                      1|
|   9574| 42|              1|                      1|
|   9586| 45|              1|                      6|
|   5032| 39|              1|                      2|
|  21850| 36|              1|                      0|
|   4711| 62|              1|                      0|
|  14487| 39|              1|                      0|
+-------+---+---------------+-----------------------+

